In [1]:
import pandas as pd
import numpy as np
import json
import pandas as pd
import re
from rdkit import Chem
from rdkit.Chem import AllChem
from collections import Counter,defaultdict
from ast import literal_eval
import cobra
from cobra import Metabolite,Reaction,Gene
from tqdm import tqdm
import random
import pickle
from multiprocessing import Pool

import sys
# sys.path.append('../Script/')
from common import *
# from check_newinfo_getyeast8U import *

# get Yeast-MetaTwin without lipids

In [2]:
### input
yeast870_path =  '../../Data/model/Yeast-MetaTwin-lipid.yml'
YMDB_total_smiles_path = '../../Data/database/ymdb/ymdb_final_result.xlsx'

cut_off = 0.3
num = 'all'
num_process = 5
cut_off_path = f'../../Results/lipid/top{num}_{cut_off}_re/'
metabolites_info_to_GEM_path = cut_off_path + f'metabolites_info_to_GEM_top{num}_{cut_off}_remain.csv'
rxndb_total_info_to_model_path = cut_off_path + f'rxndb_total_info_to_model_top{num}_{cut_off}_remain.csv'
yeast8_reaction_in_rxndb_json = cut_off_path + f'yeast8_reaction_in_rxndb_top{num}_{cut_off}_remain.json'

### output
yeast8U_withlipids_path = '../../Data/model/Yeast-MetaTwin-lipid.yml'
rxndb_total_info_to_model_reset_path = cut_off_path + f'rxndb_total_info_to_model_top{num}_{cut_off}_remain_reset.csv'

# met_info add YMDBID and YMDBname

In [3]:
met_info = pd.read_csv(metabolites_info_to_GEM_path)
met_info = met_info[~met_info['ID'].str.contains('s_')]
YMDB_Data = pd.read_excel(YMDB_total_smiles_path)
YMDB_Data = YMDB_Data[YMDB_Data['in_model']==0]

def process_metinfoID(metinfoID, met_info, YMDB_Data):
    metinfoID_smiles = met_info[met_info['ID'] == metinfoID]['new_met_smiles'].to_list()[0]
    YMDB_ID = []
    for index, row in YMDB_Data.iterrows():
        if compare_smiles_inchikey(metinfoID_smiles, row['SMILES']):
            YMDB_ID.append(row['ID'])
    return metinfoID, YMDB_ID

metinfoID_2_YMDBID = {}
metinfoID_list = list(set(met_info['ID'].to_list()))
process_partial = partial(process_metinfoID, met_info=met_info, YMDB_Data=YMDB_Data)
with mp.Pool(100) as pool:
    results = list(tqdm(pool.imap_unordered(process_partial, metinfoID_list), total=len(metinfoID_list)))
for metinfoID, YMDB_ID in results:
    metinfoID_2_YMDBID[metinfoID] = YMDB_ID

100%|██████████| 1102/1102 [09:45<00:00,  1.88it/s]


# reset rxnID

In [4]:
rxndb_to_model = pd.read_csv(rxndb_total_info_to_model_path)
rxndb_to_model['equation_dict'] = rxndb_to_model['equation_dict'].apply(lambda x:literal_eval(x))
rxndb_to_model = rxndb_to_model[~rxndb_to_model['equation'].str.startswith(' =>')]
rxndb_to_model = rxndb_to_model[~rxndb_to_model['equation'].str.endswith(' => ')]
rxndb_to_model = rxndb_to_model.drop_duplicates(['equation'], keep='first')
with open(yeast8_reaction_in_rxndb_json, 'r') as file:
    yeast8_reaction_in_rxndb_dict = json.load(file)
yeast8_reaction_in_rxndb = list(set([x for lst in yeast8_reaction_in_rxndb_dict.values() for x in lst]))
rxndb_to_model = rxndb_to_model[~rxndb_to_model['NO'].isin(yeast8_reaction_in_rxndb)]
rxndb_to_model = rxndb_to_model.reset_index(drop=True)
rxndb_to_model['NO'] = 'rxn' + (rxndb_to_model.index + 51341).astype(str)
rxndb_to_model.to_csv(rxndb_total_info_to_model_reset_path,index=None)
print(rxndb_to_model.shape)
rxndb_to_model.head(3)

(1062, 8)


,NO,EC_number,templateID,rxn_smiles_basic,rxn_smiles_final,GPR,equation_dict,equation
0,rxn51341,3.1.3,MNXR114735,CCCCCCCCCCCCCC=CC(O)C([NH3+])CO.O=P([O-])([O-]...,CCCCCCCCCCCCCC=CC(O)C([NH3+])CO.O=P([O-])([O-]...,(YDR063W) or (YDR051C) or (YKR053C) or (YIL002...,"{'sn_35': -1, 's_1322': -1, 'sn_292': 1, 's_08...",sn_35 + s_1322 => sn_292 + s_0803
1,rxn51342,3.1.3,MNXR114735,CC(C)=CCCC(C)=CCCC(C)=CCCC(C)=CCCC(C)=CCCC(C)=...,CC(C)=CCCC(C)=CCCC(C)=CCCC(C)=CCCC(C)=CCCC(C)=...,(YDR063W) or (YDR051C) or (YKR053C) or (YIL002...,"{'sl_4995': -1, 's_1322': -1, 'sl_3211': 1, 's...",sl_4995 + s_1322 => sl_3211 + s_0803
2,rxn51343,2.1.1,MNXR108123_reverse,COc1c(O)c(C)c(CC=C(C)CCC=C(C)CCC=C(C)CCC=C(C)C...,COc1c(O)c(C)c(CC=C(C)CCC=C(C)CCC=C(C)CCC=C(C)C...,(YDR497C) or (YBR236C) or (YCR024C-A) or (YKL1...,"{'s_1535': -1, 's_1413': -1, 's_0794': -1, 's_...",s_1535 + s_1413 + s_0794 => s_1416 + sn_194


In [5]:
get_yeast8U_step4(metabolites_info_to_GEM_path,rxndb_total_info_to_model_reset_path,yeast870_path,yeast8U_withlipids_path,metinfoID_2_YMDBID)

(1536, 4)
(1062, 8)


100%|██████████| 1062/1062 [00:25<00:00, 41.21it/s]


In [6]:
yeast8U_mod = cobra.io.load_yaml_model(yeast8U_withlipids_path)
yeast8U_mod.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
s_0420,r_1654,846.7,0,0.00%
s_0796,r_1832,1.029,0,0.00%
s_0925,r_1861,0.006149,0,0.00%
s_1277,r_1992,1000,0,0.00%
s_1374,r_2020,0.7108,0,0.00%
s_1438,r_2049,0.7774,0,0.00%
s_1468,r_2060,3.916,0,0.00%
s_4200,r_4593,0.2526,0,0.00%
s_4201,r_4594,0.129,0,0.00%
s_4202,r_4595,0.5346,0,0.00%
